<a href="https://colab.research.google.com/github/pseeej/SmartSystemsSoftware/blob/master/BigdataAnalysisInBiz/7%EC%A3%BC%EC%B0%A8_%EC%82%AC%EC%A0%84_%EB%85%B9%ED%99%94_%EA%B0%95%EC%9D%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

교재 : SQL로 맛보는 데이터 전처리 분석, 노수영

**실습 참고자료**

https://www.sqlitetutorial.net/
<br>https://www.w3schools.com/sql/default.asp

#실습환경 셋업

## 1) sqlite 업그레이드

In [1]:
!apt-get update
!apt-get upgrade sqlite3

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://arch

In [2]:
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.9M  100 10.9M    0     0  9020k      0  0:00:01  0:00:01 --:--:-- 9020k
/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for a sed that does not truncate output... /bin/sed
checking for grep that handles long lines and -e... /bin/grep
checking for egrep... /bin/grep -E
checking for fgrep... /bin/grep -F
checking for ld used by gcc... /usr/bin/ld
checkin

In [1]:
# 메뉴 런타임 > 런타임 다시 시작 후 셀 실행
import pandas as pd
import sqlite3
sqlite3.sqlite_version  # 3.36.0

'3.36.0'

In [2]:
!python --version

Python 3.7.12


## 2) Instacart 데이터베이스 다운로드

In [3]:
import gdown

url = 'https://drive.google.com/uc?id=17q9zfOWSRVw_H0tQn03xyLvMcy-ynPE6'
output = 'Instacart.db'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=17q9zfOWSRVw_H0tQn03xyLvMcy-ynPE6
To: /content/Instacart.db
100%|██████████| 1.07M/1.07M [00:00<00:00, 101MB/s]


'Instacart.db'

## 3) Instacart 데이터베이스 연결

In [4]:
conn = sqlite3.connect("Instacart.db")
cur = conn.cursor()

## 4) Instacart 데이터베이스 테이블 조회 테스트

In [5]:
def viewlimit(table, cutlimit):
  sql = "SELECT * FROM " + table + " LIMIT " + cutlimit + ";"
  cur.execute(sql)
  rows = cur.fetchall()
  print(sql)
  for row in rows:
    print(row)

In [6]:
viewlimit("orders","10")

SELECT * FROM orders LIMIT 10;
(2352815, 185282, 'prior', 5, 2, 9, 6)
(1414622, 41949, 'prior', 9, 0, 16, 6)
(2534064, 24631, 'prior', 66, 1, 13, 4)
(338572, 95289, 'prior', 4, 2, 14, 18)
(2578618, 170417, 'prior', 5, 1, 13, 14)
(1756647, 127210, 'prior', 31, 3, 18, 9)
(2636806, 26473, 'prior', 24, 2, 17, 17)
(681993, 89155, 'prior', 17, 1, 15, 4)
(826339, 67750, 'prior', 7, 3, 14, 17)
(57285, 89307, 'prior', 48, 1, 22, 5)


## 5) Pandas read_sql 함수로 데이터 조회 및 결과 확인

In [7]:
window_sql = f"""
select *
  from orders
"""
df = pd.read_sql(window_sql, conn, index_col=None)
df.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2352815,185282,prior,5,2,9,6
1,1414622,41949,prior,9,0,16,6
2,2534064,24631,prior,66,1,13,4
3,338572,95289,prior,4,2,14,18
4,2578618,170417,prior,5,1,13,14
5,1756647,127210,prior,31,3,18,9
6,2636806,26473,prior,24,2,17,17
7,681993,89155,prior,17,1,15,4
8,826339,67750,prior,7,3,14,17
9,57285,89307,prior,48,1,22,5


In [8]:
#db조회 결과 다운로드
df.to_csv('/content/tmp.csv')

## 6) 구글 드라이브 마운트 및 파일 저장

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


아래 코드 실행 전 내 드라이브에 result 폴더를 생성하고 해당 폴더 안에 7w 폴더를 생성함

In [10]:
def sql_result(sql,file):
  print(sql)
  df = pd.read_sql(sql, conn, index_col=None)
  print(df.head(10))
  file_name = '/content/drive/MyDrive/result/7w/' + file + '.csv' #csv파일을 MyDrive/result/7w 폴더에 저장
  print(file_name)
  df.to_csv(file_name, index=False)

In [11]:
sql = f"""
select *
  from orders
 limit 10
"""
file = 'result_7w_test'

In [12]:
sql_result(sql,file)


select *
  from orders
 limit 10

   order_id  user_id  ... order_hour_of_day  days_since_prior_order
0   2352815   185282  ...                 9                       6
1   1414622    41949  ...                16                       6
2   2534064    24631  ...                13                       4
3    338572    95289  ...                14                      18
4   2578618   170417  ...                13                      14
5   1756647   127210  ...                18                       9
6   2636806    26473  ...                17                      17
7    681993    89155  ...                15                       4
8    826339    67750  ...                14                      17
9     57285    89307  ...                22                       5

[10 rows x 7 columns]
/content/drive/MyDrive/result/7w/result_7w_test.csv


# 1.데이터 생성

데이터 다운로드 URL https://github.com/billyrohh/instacart_dataset

데이터베이스 소개 https://www.kaggle.com/c/instacart-market-basket-analysis/data

처음 접하는 데이터가 있다면 항상 각 테이블이 무엇을 의미하는지, 어떤 컬럼이 있는 확인하고 테이블 간의 관계를 파악해야 한다.

<center>

|테이블명|내용|
|-----|-----|
|aisles|상품카테고리정보1|
|departments|상품카테고리정보2|
|order_products__prior|각 주문 번호의 상세 구매 내역|
|orders|주문 대표 정보|
|products|상품정보|
</center>

[aisles]
*   aisle_id
*   aisle

[departments]
*   department_id
*   department

[order_products__prior]
*   order_id
*   product_id
*   add_to_cart_order
*   reordered

[orders]
*   order_id
*   user_id
*   eval_set
*   order_number
*   order_dow
*   order_hour_of_day
*   days_since_prior

[products]
*   product_id
*   product_name
*   aisle_id
*   department_id

# 4.구매자 분석 (사전녹화)

### 1) 10분위 분석

*   10분위 분석을 통해서 서비스 주문 수가 VIP 고객에게 얼마나 집중되어 있는지 확인함

*   고객별로 주문 건수를 계산한 뒤, 주문 건수를 기준으로 각 고객이 어떤 그룹에 속하는지 구함

*   다음으로 각 분위 위수별 주문 건수의 합을 구하면, 전체 주문건이 어떤 그룹에 얼마나 집중되어 있는지 계산할 수 있음

<center>

|그룹|비중|
|---|---|
|1|0~10%|
|2|10~20%|
|3|20~30%|
|4|30~40%|
|5|40~50%|
|6|50~60%|
|7|60~70%|
|8|70~80%|
|9|80~90%|
|10|90~100%|

</center>

*10분위 분석이란?

*   전체를 10분위로 나누어 각 분위 수에 해당하는 집단의 성질을 나타내는 방법.

*   10분위 분석을 진행하려면 먼저 각 구매자의 분위 수를 구해야 함
*   고객들의 주문 건수를 기준으로 분위 수를 나눔

*   STEP1 : 먼저 다음과 같은 방법으로 주문 건수에 따른 Rank를 생성함

In [13]:
# 고객별 주문 건수 기준 랭킹
sql = f"""
SELECT *
     , ROW_NUMBER() OVER (ORDER BY F DESC) RNK
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A
"""
sql_result(sql,'4_1_1구매자분석')


SELECT *
     , ROW_NUMBER() OVER (ORDER BY F DESC) RNK
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A

   USER_ID  F  RNK
0    32099  3    1
1     2610  2    2
2    10132  2    3
3    10972  2    4
4    12435  2    5
5    22379  2    6
6    25449  2    7
7    33244  2    8
8    37499  2    9
9    37801  2   10
/content/drive/MyDrive/result/7w/4_1_1구매자분석.csv


* STEP2 : 고객별로 분위 수를 매기려면 전체 고객이 몇 명인지 알아야 함
*   다음과 같은 방법으로 전체 고객 수를 계산함

In [14]:
# 전체 고객수
sql = f"""
SELECT COUNT(DISTINCT USER_ID)
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A
"""
sql_result(sql,'4_1_2구매자분석')


SELECT COUNT(DISTINCT USER_ID)
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A

   COUNT(DISTINCT USER_ID)
0                     3159
/content/drive/MyDrive/result/7w/4_1_2구매자분석.csv


*   STEP3 : 전체 고객 수는 3,159명으로 등수별 분위 수 는 다음과 같이 계산함

*   각 등수에 따른 분위 수는 CASE WHEN 구문을 이용해 다음과 같은 방법으로 설정할 수 있음

In [15]:
#전체 고객수 기준 
sql = f"""
-- 고객별 주무 건수 기준 랭킹 분위수 단위 변경
SELECT *
     , CASE WHEN RNK BETWEEN 1    AND 316  THEN 'Quantile_1' --316명
            WHEN RNK BETWEEN 317  AND 632  THEN 'Quantile_2'
            WHEN RNK BETWEEN 633  AND 948  THEN 'Quantile_3'
            WHEN RNK BETWEEN 949  AND 1264 THEN 'Quantile_4'
            WHEN RNK BETWEEN 1265 AND 1580 THEN 'Quantile_5'
            WHEN RNK BETWEEN 1581 AND 1896 THEN 'Quantile_6'
            WHEN RNK BETWEEN 1897 AND 2212 THEN 'Quantile_7'
            WHEN RNK BETWEEN 2213 AND 2528 THEN 'Quantile_8'
            WHEN RNK BETWEEN 2529 AND 2844 THEN 'Quantile_9'
            WHEN RNK BETWEEN 2845 AND 3159 THEN 'Quantile_10' --315명
        END quantile

-- 고객별 주문 건수 기준 랭킹
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F DESC) RNK
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A
     ) A
;
"""
sql_result(sql,'4_1_3구매자분석')


-- 고객별 주무 건수 기준 랭킹 분위수 단위 변경
SELECT *
     , CASE WHEN RNK BETWEEN 1    AND 316  THEN 'Quantile_1' --316명
            WHEN RNK BETWEEN 317  AND 632  THEN 'Quantile_2'
            WHEN RNK BETWEEN 633  AND 948  THEN 'Quantile_3'
            WHEN RNK BETWEEN 949  AND 1264 THEN 'Quantile_4'
            WHEN RNK BETWEEN 1265 AND 1580 THEN 'Quantile_5'
            WHEN RNK BETWEEN 1581 AND 1896 THEN 'Quantile_6'
            WHEN RNK BETWEEN 1897 AND 2212 THEN 'Quantile_7'
            WHEN RNK BETWEEN 2213 AND 2528 THEN 'Quantile_8'
            WHEN RNK BETWEEN 2529 AND 2844 THEN 'Quantile_9'
            WHEN RNK BETWEEN 2845 AND 3159 THEN 'Quantile_10' --315명
        END quantile

-- 고객별 주문 건수 기준 랭킹
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F DESC) RNK
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A
     ) A
;

   USER_ID  F  RNK    quantile
0    32099  3    1  Quantile_1
1     2610  2    2  Quantile_1
2    10132  2    3  Quantile_1
3 

*   참고 : CASE WHEN 구문에서는 각 분위 수별로 등수를 지정했는데, 모든 등수를 기재할 필요 없이 다음과 같은 방법으로도 처리 가능함

In [16]:
# 위랑 동일한 결과로 나옴

sql = f"""
SELECT *
     , CASE WHEN RNK <= 316  THEN 'Quantile_1'
            WHEN RNK <= 632  THEN 'Quantile_2'
            WHEN RNK <= 948  THEN 'Quantile_3'
            WHEN RNK <= 1264 THEN 'Quantile_4'
            WHEN RNK <= 1580 THEN 'Quantile_5'
            WHEN RNK <= 1896 THEN 'Quantile_6'
            WHEN RNK <= 2212 THEN 'Quantile_7'
            WHEN RNK <= 2528 THEN 'Quantile_8'
            WHEN RNK <= 2844 THEN 'Quantile_9'
            WHEN RNK <= 3159 THEN 'Quantile_10'
        END quantile
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F DESC) RNK
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A
     ) A
;
"""
sql_result(sql,'4_1_4구매자분석')


SELECT *
     , CASE WHEN RNK <= 316  THEN 'Quantile_1'
            WHEN RNK <= 632  THEN 'Quantile_2'
            WHEN RNK <= 948  THEN 'Quantile_3'
            WHEN RNK <= 1264 THEN 'Quantile_4'
            WHEN RNK <= 1580 THEN 'Quantile_5'
            WHEN RNK <= 1896 THEN 'Quantile_6'
            WHEN RNK <= 2212 THEN 'Quantile_7'
            WHEN RNK <= 2528 THEN 'Quantile_8'
            WHEN RNK <= 2844 THEN 'Quantile_9'
            WHEN RNK <= 3159 THEN 'Quantile_10'
        END quantile
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F DESC) RNK
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A
     ) A
;

   USER_ID  F  RNK    quantile
0    32099  3    1  Quantile_1
1     2610  2    2  Quantile_1
2    10132  2    3  Quantile_1
3    10972  2    4  Quantile_1
4    12435  2    5  Quantile_1
5    22379  2    6  Quantile_1
6    25449  2    7  Quantile_1
7    33244  2    8  Quantile_1
8    37499  2    9  Quantile_1
9    37801  2   10 

## 2) 분위수 별 특성 파악

*   각 분위 수별 특성을 파악해 보고자 함
*   각 분위 수별로 평균 주문수를 파악함
*   STEP1 : 먼저 위의 조회 결과를 하나의 테이블로 생성해 user_id별 분위 수 정보를 생성함

In [17]:
#만약 USER_QUANTILE 테이블이 존재하는 경우 삭제
sql = f"""
DROP TABLE IF EXISTS USER_QUANTILE;
"""
conn.execute(sql)

In [18]:
sql = f"""

CREATE TABLE USER_QUANTILE AS
SELECT *
     , CASE WHEN RNK <= 316  THEN 'Quantile_1'
            WHEN RNK <= 632  THEN 'Quantile_2'
            WHEN RNK <= 948  THEN 'Quantile_3'
            WHEN RNK <= 1264 THEN 'Quantile_4'
            WHEN RNK <= 1580 THEN 'Quantile_5'
            WHEN RNK <= 1896 THEN 'Quantile_6'
            WHEN RNK <= 2212 THEN 'Quantile_7'
            WHEN RNK <= 2528 THEN 'Quantile_8'
            WHEN RNK <= 2844 THEN 'Quantile_9'
            WHEN RNK <= 3159 THEN 'Quantile_10'
        END quantile
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F DESC) RNK
  FROM (
SELECT USER_ID
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
     ) A
     ) A
;
"""
conn.execute(sql)

In [19]:
#테이블 리스트 확인
sql = f"""
SELECT name FROM sqlite_master WHERE type='table';
"""
df = pd.read_sql(sql, conn, index_col=None)
df

,name
0,aisles
1,departments
2,order_products__prior
3,orders
4,products
5,USER_QUANTILE


*   STEP2 : 다음과 같은 방법으로 각 분위 수별 전체 주문 건수의 합을 구할 수 있음

In [23]:
# SUM(F)는 구매한 사람들의 수...?
# quantile 이름 순으로 정렬함 (order by 1 desc)

sql = f"""
SELECT QUANTILE
     , SUM(F) F
  FROM USER_QUANTILE
 GROUP BY 1
 ORDER BY 1 DESC
;
"""
sql_result(sql,'4_1_5구매자분석')


SELECT QUANTILE
     , SUM(F) F
  FROM USER_QUANTILE
 GROUP BY 1
 ORDER BY 1 ASC
;

      quantile    F
0   Quantile_1  377
1  Quantile_10  315
2   Quantile_2  316
3   Quantile_3  316
4   Quantile_4  316
5   Quantile_5  316
6   Quantile_6  316
7   Quantile_7  316
8   Quantile_8  316
9   Quantile_9  316
/content/drive/MyDrive/result/7w/4_1_5구매자분석.csv


*   STEP3 : 전체 주문 건수를 계산하고, 각 분위 수의 주문 건수를 전체 주문 건수로 나누기 함

In [24]:
#전체 주문 건수
sql = f"""
SELECT SUM(F)
  FROM USER_QUANTILE
;
"""
sql_result(sql,'4_1_6구매자분석')


SELECT SUM(F)
  FROM USER_QUANTILE
;

   SUM(F)
0    3220
/content/drive/MyDrive/result/7w/4_1_6구매자분석.csv


In [25]:
#분위수별 주문건수 / 전체 주문건수
sql = f"""
SELECT QUANTILE
     , SUM(F)/(3220 +0.0) F
  FROM USER_QUANTILE
 GROUP BY 1
;
"""
sql_result(sql,'4_1_7구매자분석')


SELECT QUANTILE
     , SUM(F)/(3220 +0.0) F
  FROM USER_QUANTILE
 GROUP BY 1
;

      quantile         F
0   Quantile_1  0.117081
1  Quantile_10  0.097826
2   Quantile_2  0.098137
3   Quantile_3  0.098137
4   Quantile_4  0.098137
5   Quantile_5  0.098137
6   Quantile_6  0.098137
7   Quantile_7  0.098137
8   Quantile_8  0.098137
9   Quantile_9  0.098137
/content/drive/MyDrive/result/7w/4_1_7구매자분석.csv


*   결과 : 각 분위 수별로 주문 건수가 거의 균등하게 분포되어 있음
*   해당 서비스는 매출이 VIP에게 집중되지 않고, 전체 고객에게 고르게 분포되어 있음을 알 수 있음

# 5.상품 분석 (사전녹화)

*   재구매를 많이 하는 상품을 알아보고 각 상품의 판매 특성에 대해 확인

*   재구매 비중이 높은 상품을 찾아봄
*   STEP1 : 상품별 재구매 비중(%)과 주문 건수를 계산함

In [26]:
#상품별 재구매 비중과 주문 건수 계산
# order by product_id로 했으므로 "상품별"로 됨

sql = f"""
SELECT PRODUCT_ID
     , SUM(REORDERED+0.0)/COUNT(*) REORDER_RATE
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY PRODUCT_ID
 ORDER BY REORDER_RATE DESC
;
"""
sql_result(sql,'5_1상품분석')


SELECT PRODUCT_ID
     , SUM(REORDERED+0.0)/COUNT(*) REORDER_RATE
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY PRODUCT_ID
 ORDER BY REORDER_RATE DESC
;

   product_id  REORDER_RATE  F
0       49682           1.0  1
1       49680           1.0  1
2       49678           1.0  1
3       49677           1.0  1
4       49638           1.0  1
5       49630           1.0  1
6       49615           1.0  2
7       49587           1.0  1
8       49566           1.0  1
9       49560           1.0  1
/content/drive/MyDrive/result/7w/5_1상품분석.csv


*   STEP2 : 주문 건수가 1건인 경우가 많으므로, 주문 건수가 일정 건수(10건) 이하인 상품은 제외함
*   HAVING을 이용하면 일정 건수 이하인 상품을 쉽게 제외할 수 있음

In [27]:
# having은 group by로 설정한 거에 조건 두고 싶을 때 사용

sql = f"""
SELECT A.PRODUCT_ID
     , SUM(REORDERED+0.0)/COUNT(*) REORDER_RATE
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR A
  LEFT JOIN PRODUCTS B ON A.PRODUCT_ID = B.PRODUCT_ID
 GROUP BY A.PRODUCT_ID
HAVING COUNT(DISTINCT ORDER_ID) > 10
;
"""
sql_result(sql,'5_2상품분석')


SELECT A.PRODUCT_ID
     , SUM(REORDERED+0.0)/COUNT(*) REORDER_RATE
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR A
  LEFT JOIN PRODUCTS B ON A.PRODUCT_ID = B.PRODUCT_ID
 GROUP BY A.PRODUCT_ID
HAVING COUNT(DISTINCT ORDER_ID) > 10
;

   product_id  REORDER_RATE   F
0          45      0.777778  18
1         196      0.685714  35
2         260      0.724138  29
3         432      0.810811  37
4         651      0.615385  13
5         890      0.500000  24
6        1158      0.600000  15
7        1215      0.769231  13
8        1244      0.454545  11
9        1463      0.857143  28
/content/drive/MyDrive/result/7w/5_2상품분석.csv


*HAVING vs WHERE

*SQL을 처음 배울 때 HAVING과 WHERE의 차이점에 대해 혼란스러워 하는 경향이 있다. 2가지 구문의 차이점을 무엇일까?*

[WHERE]
*   WHERE 절은 FROM에 위치한 테이블에만 조건을 걸 수 있음
*   SELECT문에서 새롭게 생성한 칼럼에 조건을 걸어야 하는 경우가 있는데, SELECT에서 새롭게 생성한 칼럼을 WHERE에서 사용하면 명령이 실행되지 않음

[HAVING]
*   HAVING은 그룹핑(GROUP BY)한 데이터에 조건을 생성하고 싶을 때 사용함

[HAVING 예시]
*   PRODUCT_ID로 데이터를 그룹핑한 뒤, 주문 번호를 카운트함 
*   그룹핑한 데이터에 조건을 생성하고 싶은 경우 HAVING을 이용함
*   쿼리를 실행하면 주문 건수가 10보다 큰 데이터만 출력됨

In [30]:
# 상품별로 주문이 10건 이상인 상품을 추출할 때 HAVING을 사용함
# 위에서 count(distinct order_id)를 f로 뒀지만~ 이건 그냥 표현할 때만 이렇게 쓰는 거라
# 아래에서 having count(distinct order_id) > 10에서 똑같은 내용을 다시 한 번 쓰나봐...

sql = f"""
SELECT A.PRODUCT_ID
     , B.PRODUCT_NAME
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR A
  LEFT JOIN PRODUCTS B
    ON A.PRODUCT_ID = B.PRODUCT_ID
 GROUP BY A.PRODUCT_ID
     , B.PRODUCT_NAME
HAVING COUNT(DISTINCT ORDER_ID) > 10
;
"""
sql_result(sql,'5_3HAVING')


SELECT A.PRODUCT_ID
     , B.PRODUCT_NAME
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR A
  LEFT JOIN PRODUCTS B
    ON A.PRODUCT_ID = B.PRODUCT_ID
 GROUP BY A.PRODUCT_ID
     , B.PRODUCT_NAME
HAVING COUNT(DISTINCT ORDER_ID) > 10
;

   product_id                            product_name   F
0          45                       European Cucumber  18
1         196                                    Soda  35
2         260                              Cantaloupe  29
3         432       Vanilla Almond Breeze Almond Milk  37
4         651                   Organic Salted Butter  13
5         890                  Organic Diced Tomatoes  24
6        1158                            Mango Chunks  15
7        1215  Kidz All Natural Baked Chicken Nuggets  13
8        1244              Organic Orange Bell Pepper  11
9        1463                            Organic Milk  28
/content/drive/MyDrive/result/7w/5_3HAVING.csv


*   STEP3 : 어떤 상품들이 재구매율이 높은지 보기 위해 PRODUCTS 테이블을 Join해 살펴봄

In [31]:
sql = f"""
SELECT A.PRODUCT_ID
     , B.PRODUCT_NAME
     , SUM(REORDERED+0.0)/COUNT(*) REORDER_RATE
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR A

-- 상품 데이터 추가
  LEFT JOIN PRODUCTS B
    ON A.PRODUCT_ID = B.PRODUCT_ID

 GROUP BY A.PRODUCT_ID
     , B.PRODUCT_NAME
HAVING COUNT(DISTINCT ORDER_ID) > 10
;
"""
sql_result(sql,'5_4상품분석')


SELECT A.PRODUCT_ID
     , B.PRODUCT_NAME
     , SUM(REORDERED+0.0)/COUNT(*) REORDER_RATE
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDER_PRODUCTS__PRIOR A

-- 상품 데이터 추가
  LEFT JOIN PRODUCTS B
    ON A.PRODUCT_ID = B.PRODUCT_ID

 GROUP BY A.PRODUCT_ID
     , B.PRODUCT_NAME
HAVING COUNT(DISTINCT ORDER_ID) > 10
;

   product_id                            product_name  REORDER_RATE   F
0          45                       European Cucumber      0.777778  18
1         196                                    Soda      0.685714  35
2         260                              Cantaloupe      0.724138  29
3         432       Vanilla Almond Breeze Almond Milk      0.810811  37
4         651                   Organic Salted Butter      0.615385  13
5         890                  Organic Diced Tomatoes      0.500000  24
6        1158                            Mango Chunks      0.600000  15
7        1215  Kidz All Natural Baked Chicken Nuggets      0.769231  13
8        1244              Organic Ora